In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [2]:
client_id = 'dd1e7da8e4924986931df06f52c0ab26'
client_secret = '93c91e7a06df4ca6b4984ebbb0c5e891'

In [9]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__ (self,client_id,client_secret, *args,**kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" 
        } 
    def get_token_data(self):
        return {
            "grant_type":"client_credentials"
        } 

    def perform_auth(self): #method for getting access token
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers= token_headers)
        if r.status_code not in range (200,299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now() #gives the exact time the request is happening
        access_token = data['access_token']
        expires_in =  data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)#tells you how many seconds your access token expires in so you know when you renew your access_token
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now  #gives boolean response with whether or not token is expired
        return True
    
    def get_access_token(self):
        #auth_done = self.perform_auth()
        #if not auth_done:
         #   raise Exception("Authentication failed")
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version = 'v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return{}
        return r.json()
        
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    
    def base_search(self, query_params): #type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception ("A query is required")
        if isinstance(query,dict):
                query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query,str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q":query, "type": search_type.lower()}) ##encodes spaces
        print(query_params)
        return self.base_search(query_params)
        


In [10]:
spotify = SpotifyAPI(client_id, client_secret)

In [11]:
spotify.search({"track":"A lannister always pays his debts", "artist":"Ramin"}, search_type="artist")

q=track%3AA+lannister+always+pays+his+debts+artist%3ARamin&type=artist
https://api.spotify.com/v1/search?q=track%3AA+lannister+always+pays+his+debts+artist%3ARamin&type=artist


{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3AA+lannister+always+pays+his+debts+artist%3ARamin&type=artist&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [15]:
spotify.search(query = "Danger Zone", search_type="track")

q=Danger+Zone&type=track
https://api.spotify.com/v1/search?q=Danger+Zone&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Danger+Zone&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Y3xIwWyq5wnNHPp5gPjOW'},
       'href': 'https://api.spotify.com/v1/artists/3Y3xIwWyq5wnNHPp5gPjOW',
       'id': '3Y3xIwWyq5wnNHPp5gPjOW',
       'name': 'Kenny Loggins',
       'type': 'artist',
       'uri': 'spotify:artist:3Y3xIwWyq5wnNHPp5gPjOW'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'I

In [ ]:
#spotify.get_artist("1hCkSJcXREhrodeIHQdav8")

In [ ]:
#access_token = spotify.access_token

In [ ]:
headers = {
    "Authorization":f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q":"time", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

In [ ]:
r.json()

In [ ]:
data = urlencode({"q":"A Lannister Always pays his debts", "type": "track"})
lookup_url = f"{endpoint}?{data}"
r = requests.get(lookup_url, headers=headers)
r.json()

In [ ]:
#client_creds = f"{client_id}:{client_secret}"
#type(client_creds)

In [ ]:
#client_creds.encode() #turn str into bytes, .decode() turns it back

In [ ]:
#client_creds_b64 = base64.b64encode(client_creds.encode()) #encodes credentials and changes type to bytes
#type(client_creds_b64)

In [ ]:
#token_url = "https://accounts.spotify.com/api/token"
#method = "POST"
#token_data = {
 #   "grant_type":"client_credentials"
#}
#token_headers = {
 #   "Authorization": f"Basic {client_creds_b64.decode()}" #<base64 encoded client_id:client_secret>
#}


In [ ]:
#r = requests.post(token_url, data=token_data, headers= token_headers)
#print(r.json())
#valid_request = r.status_code in range (200,299)